# Motion estimation

In this section we will go through how to perform motion analysis on MPS data. The material here is mostly based on the documentation at https://computationalphysiology.github.io/mps_motion

First we need to import `mps` for reading the data and `mps_motion` for performing motion analysis

In [ ]:
from pathlib import Path # For working with file paths
import mps  # For reading MPS data
import mps_motion # For motion estimation
import matplotlib.pyplot as plt # For plotting

## Loading data

Now load the data 

In [ ]:
motion_data_path = Path("motion_data.npy")

def download_data():
    print("Downloading data. Please wait...")
    link = "https://www.dropbox.com/s/kh6jlvyjyxpcea3/motion_data.npy?dl=1"
    import urllib.request
    import time

    urllib.request.urlretrieve(link, motion_data_path)
    time.sleep(1.0)
    print("Done downloading data")
    
if not motion_data_path.is_file():
    download_data()
    
data = mps.MPS(motion_data_path)
data

Let us first print some info about the data

In [ ]:
data.info

Notice in particular that we have 0.65 micro meter per pixel. This mean that the physical width and height of one pixel is 0.65 micrometer. This information will be important when converting the displacment and velocity to real units.

We can also display convert the frames to a video file that we can play in the notebook

In [ ]:
movie_path = "motion_data.mp4"
mps.utils.frames2mp4(data.frames.T, movie_path, framerate=data.framerate)

In [ ]:
from IPython.display import Video
Video(movie_path)

## Displacement and velocity

We will discuss two different measures of motion, namely the velocity and the displacement. These things are tightly related. 

We will refer to the *displacement* as the motion from one specific reference image. In other words, we define one image in the recording as the reference image and for all the other images, we compute the motion from this reference image to every othere image. 

The *velocity* on the other had has reference image that depends on the current image, so that the velocity at a given point in time is defined as the motion from a previous image with a fixed number of images before it. We will refer to this number as the spacing. For example, we could choose a spacing of 5, in which case the velocity at a given point will be the motion of the image five timepoints before it. 

## Creating optical flow object

Before starting we will create an optical flow object which is the object we use to run the motion tracking software. Here we have chosen the Farneback optical flow algorithm

In [ ]:
opt_flow = mps_motion.OpticalFlow(data, flow_algorithm="farneback")

To list available optical flow algorithms you can use

In [ ]:
mps_motion.list_optical_flow_algorithm()

## Estimating the velocity and reference frame

Before we can run the motion analysis we need to estimate a suitable reference frame. We can do this by first estimate the velocity (let us use a spacing of 5)

In [ ]:
spacing = 5
v = opt_flow.get_velocities(spacing=spacing)
print(v)

We see that the object we get back is a `VectorFrameSequence`. This is a special object that represents a vector field for each image in the sequence of images, and we see that is has dimension (number of pixels in $\times$ number of pixels in $\times$ number of time steps  2) where the final two dimensions are the and component of the vectors. The default units of time is milliseconds, so velocity has units micrometer per miliseconds. Let us convert this to micro meter per seconds

In [ ]:
v *= 1000

We can now compute the velocity norm and average over the image

In [ ]:
v_norm_mean = v.norm().mean().compute()

Notice that we need to also call `compute()` on the object. This is becase the compuation will be executed in paralell. 

Let us now use the velocity to estimate the reference frame. This algorithm will use the the zero velocity baseline a find a frame where the velocity is zero. We must also provide the time stamps with the same length as the velocity trace

In [ ]:
reference_frame_index = (
    mps_motion.motion_tracking.estimate_referece_image_from_velocity(
        t=data.time_stamps[:-spacing],
        v=v_norm_mean,
    )
)
reference_frame = data.time_stamps[reference_frame_index]
print(f"Found reference frame at index {reference_frame_index} and time {reference_frame:.2f}")

Let us also plot the velocity trace and mark the point where the reference frame is chosen

In [ ]:
fig, ax = plt.subplots()
ax.plot(data.time_stamps[:-spacing], v_norm_mean)
ax.plot([reference_frame], [v_norm_mean[reference_frame_index]], "ro")
plt.show()

### Exercise

Try to use a different spacing and see how this changes the results

## Computing displacement

We can now run the optical flow algorithm to extract the displacements.

In [ ]:
u = opt_flow.get_displacements(reference_frame=reference_frame)
print(u)

Similar to the velocity, the displacment is a `VectorFrameSequence`. If we take the norm of this VectorFrameSequence we get a `FrameSequence`

In [ ]:
u_norm = u.norm()
u_norm

In stead of being a `VectorFrameSequence` we now have a `FrameSequence`, so that for each value represents the norm of the vector from the `VectorFrameSequence`.

We can for example compute the maximum norm over all time steps by calling `.max()`

In [ ]:
u_norm_max = u_norm.max().compute()
print(u_norm_max.shape)

Notice that the shape of this array is the same shape as each image. And the values at each pixel will be the maximum displacment over all time points. We can plot this as an image

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
im = ax.imshow(u_norm_max.T)
cbar = fig.colorbar(im)
cbar.set_label("Maximum displacement [\u00B5m]")
plt.show()

This image will highlight regions with high displacement, and we could use it to identify the pixels with cells.

Similar to velocity, we can also compute the mean over the entire image, in order to reduce each image down to a single number.

In [ ]:
u_norm_mean = u_norm.mean().compute()

fig, ax = plt.subplots()
ax.plot(data.time_stamps, u_norm_mean)
plt.show()

One thing you might notice is that the maximum average displacement is much lower than the maximum displacement at all pixels which was closer to 5 micro meters

### Exercise

Try to use a different reference index (i.e not the one estimated) and see how this changes the results

## Computing Features

Let us now extract one beat and compute some simple feaures of the displacement and velocity. 

We have created a little utility function for plotting these features

In [ ]:
from utils import plot_features

fig, ax = plot_features(u_norm=u_norm_mean, v_norm=v_norm_mean, time=data.time_stamps)
plt.show()

We can now compute these features for the three beats

In [ ]:
features = mps_motion.stats.compute_features(u=u_norm_mean, v=v_norm_mean, t=data.time_stamps)
for key, values in features.items():
    print(key, values)